# Race Video Rendering

The Garmin Virb 360 is, on paper, is a pretty cool camera.  The Virb captures 360
video, and GPS, as well as accelerometer/gyro/magnetometer data.  (And if you have
a Garmin marine instrument package, the Virb can connect and capture instrument
data as well.)

All this cool data can be combined in a program called Virb Edit, which can stabilize
the camera viewpoint and overlay instrument data (like course map, wind speed,
etc).  Virb Edit then generates a new stabilized and "overlayed" video which can be uploaded to Youtube (which understands 360 video) and cool videos result.

Here is a screen capture created from Virb Edit that shows the idea:

![im](Data/Images/one_life_small.jpg)

**I believe annotated videos can be a great training aid.**

- With a 360 camera you can see the most of the sail, including the top, to understand sail settings. - You can pan and zoom the video so you can see around the boat, including marks and competitors.
- You can synchronize instrument data and crew actions.  This allows you to understand what caused changes in performance.
  - We could potentially overlay virtual "sensors", which can show VMG and speed versus polars.

**The problem is that Virb Edit sucks.**  It crashes (on my Mac) constantly.  It is very,
very slow.  It does not include the marine instruments (since I have B&G).

I set off on a long process to reverse engineer this whole thing, so that I could
produce my own videos, and along the way discovered there was something much, much
better that can be done.

## What is camera stabilization?

The VIRB camera may rotate in the world (roll, pitch, and yaw), which can quicky make
the video hard to view and understand.  If you knew the rotation of the camera in the world you can counter-rotate the video image,
stabilizing the view.  For example, if the camera is rolling and pitching, one can rotate the viewpoint so that "video down"
is always "true down".  Complete stabilization would fix north/east as well as down. In practice it is often better, when affixed to a vehicle, to follow the direction of travel.  It makes more sense to track along the with the boat, rather than look at a fixed compass angle.  

Finally, perfect stabilization can remove the sense of motion which is critical to understanding what is going on.  Is the boat pitching and rolling through the waves?  Often a better option is to remove the rapid shaking (high frequency) movements while preserving large smooth motions.

Quick video example of the side by side before and after [Virb Edit stabilization](https://youtu.be/uY1P2SrF0TQ?t=183).


## Key steps to replacing Virb Edit:

- Read the Garmin FIT files which contain the GPS/accel/gyro data.  
  - I started off doing this in Python, but that
  was slow.  
  - Eventually used the official C++ code. https://developer.garmin.com/fit/overview/ (full SDK there)
  - The C++ code is in FIT_SDK

- Understand the FIT file data stream.
  - The schema is here: `fit_sdk_schema.xlsx`.  It is helpful but still mysterious.
    - The accel/gyro data is stored in a odd way, and must be reinterpreted to get the
      true sensor data.
    - The C++ code also had some additional clues in how to process accel/gyro
  - The time alignment (sync) between the FIT data and the videos is not straightforward.  The FIT file includes an event when the video is created.  The video creation times are not quite identical.

- Estimate the orientation of the camera in the world using the accel/gyro. I built a 
  complementary filter.

- Take the equirectagular video captured by the Virb and view it.  Extend this to
  include the camera orientation so that the camera viewpoint can be stabilized.

- Sync the video/pose data with instrument data (captured on the boat using a
  Raspberry PI).

- Display the synchronized instrument data on the video.

At this point we would have parity with Virb Edit (great!). The plan was that after each race I would
pull the video data off the camera, reprocess to create a new video of the race with overlaid instrument data, and then upload to Youtube.  Unlike Virb Edit, all this could be done using scripts, no pesky app to run with buttons to click and questions to answer (with hours wasted!).  And this could be done alongside the processing of instrument data, so that key events (tacks/jibes/starts) could be highlighted or selected.

Along the way I discovered I could do much better.  Rather than re-processing the
VIRB video to overlay the instruments, I could create a web page that accessed the original video, camera
orientation, and sensor data.  The web page would then render, stabilize, and show
the instrument data.

Why is this a good idea?  The web browser is the most powerful and stable
video player (Netflix and Youtube use the browser video player).  And it is easily programmable.  The browser can do
all the work (very efficiently) and operate with the original unmodified video data
(no transcoding).

**Benefits:**
- We can change the display of the instruments at any time, or at a keystroke.
  - The instrument data is not embedded in the video.  It can display anywhere.
- We can change the type of camera or stabilization.
- Provide our own scheme for navigating the video and data, so you can program the video play back.
  - Hit a key to show the start.
  - Hit a key to skip to next tack.
  - Only show the interesting stuff (tacks/jibes/roundings).
  - Allow other folks to add comments, or create a play list of short segments.
  - Highlight areas where things went well/poorly (VMG?)
- The web/html/javascript has 1000's of visualization tools.  E.G. is is not hard to show NOAA
  charts alongside the video.
- Can work with other types of video (GoPro?), and multiple resolutions.

**Costs:**
- I need to host my own video (though perhaps it could come from Google).  Currently
  using S3.
- I need to create and host the web site.
- I need to deal with access control and security.
- It is not free (though time is not free!).

### Screenshot Virb Video Player

I've pulled most of the pieces together and here is a screenshot:

![im](Data/Images/virb_video.png)

## Quick note on the Virb 360

Garmin makes a 360 camera called the [Garmin Virb 360](https://buy.garmin.com/en-US/US/p/562010).  While it seems a bit dated compared with GoPro Max and more expensive, it has many advantages:

- It is a 360 camera.
- It seems like a rugged package, with some caveats:
  - There is visible internal condensation that collects on the lens.
  - GPS seems to be unreliable.
- It has a well designed external power system built into its tripod mount (ensuring I can record for 6+ hours).  My GoPro does **not** (nor does the newer GoPro Max).
- It has an idiot proof mechanism for recording video (a big red mechanical switch).  No more pushing buttons and hoping that video is recorded.  I've lost videos on the GoPro because it is tricky to get started.

If your boat has Garmin electronics the overall end to end sounds good: pair the camera with your plotter, record video, load into Virb Edit, and create videos with overlayed gauges.  It might be nice to include other data, but there is no obvious way to include information about VMG or polars. Checkout this [video from One Life](https://youtu.be/wumJrJFCvow) (instruments are a bit off to the left).

Originally I was hoping to create a FIT file from my boat instrument data and *sneak* it into Virb Edit.  But no luck.  More info here: [Garmin Fit Files](Garmin_Fit_files.ipynb).

# What is 360 Video?

Turns out that "360 video" is just a video file (e.g MP4 video) that have been stitched (or transformed) into a [equirectangular](https://en.wikipedia.org/wiki/Equirectangular_projection) image (ER for short).

Most 360 cameras are built as a pair of 180+ deg fisheye cameras. Together they capture a pair of half-sphere images that are then warped and stitched into a single image.  Note, both of these images are from [Paul Bourke's Website](http://paulbourke.net/dome/).

![im](Data/Images/two_spherical.jpg)

![im](Data/Images/equirectangular.jpg)

ER videos can be updloaded to Youtube or Facebook.  If the file tags are set correctly then many viewers will automatically operate in 360 mode.  This mode remaps a small portion of the ER image onto the screen of the user (being careful to minimize distortions).

The default mode for the Virb 360 is to capture the two spherical images and stitch them into a single ER video.  There is also a raw mode that directly captures the spherical images at higher resolution (using more data, and ominously includes warning about the camera overheating).

## Tricky bits of 360 video

There are two things that are important for a good 360 experience:

- Video stabilization
  - Action cameras can shake and this disturbs the video.
  - There are two ways to stabilize: 
    - using accel/gyros in the camera (Virb) 
    - tracking points in the video
  - See [video](https://www.youtube.com/watch?v=uY1P2SrF0TQ) for how this can work on the Virb.
  - Point tracking stabilization is a feature or After Effects and Final Cut Pro.
- Straightening the horizon
  - If the horizon is not level, then as the view pans left and right the visual motion include up and down motion *and* rotation.  Its weird.
  - There are two ways to fix this: i) using accelerometers in the camera to measure the direction of gravity (Virb) and ii) by hand.
  - See [video](https://youtu.be/GZYaGR6KRe8?t=23) for examples of how this can go wrong and how to fix it by hand.

This is an example of a rotated horizon (from the video above).

![im](Data/Images/bad_horizon_360.png)


## Equirectangular Projection

All cameras capture a portion of the view sphere.  The view sphere represents the color/brightness you would see if you looked out along rays that project from the center of the sphere.  

![im](Data/Images/view_sphere_2.gif)


The Virb 360 captures the entire view sphere through two fisheye cameras.  The {x, y} pixel coordinates in the fisheye images can be converter to spherical coordinates (called {pan, tilt} here, but are more typically called {azimuth, elevation} or equivalently {longitude, latitude}).  An equirectangular image stores the pixel value at a particular lon/lat in the pixel coordinates as follows.

\begin{align}
 x &= R_x \lambda & \quad & \text{longitude, or elevation} \\
 y &= R_y \phi    & \quad & \text{latitiude, or } \\
\end{align}

Note, an ER image is simply a way to take the pixels from a sphere and copy them into an a rectangular image, since a sphere is not easily encoded as a simple image.  The ER image is very inefficient, dedicating a lot of pixels to "north and south poles". A much better way, used by game engines and Google, is a [cubemap](https://en.wikipedia.org/wiki/Cube_mapping). Unfortunately most cameras (including the Virb360) and most players use equirectangular.  

![im](Data/Images/cubemaps_skybox.png)


### 360 Video References

- [360 Video Projection](https://en.wikipedia.org/wiki/360_video_projection)
- [Cube mapping](https://en.wikipedia.org/wiki/Cube_mapping)
  - Alternative to ER (more uniform use of pixels)
- [Google Equal Angle Cube Map](https://blog.google/products/google-vr/bringing-pixels-front-and-center-vr-video/)
  - Perhpas better than cube map.

## Conversion to a camera image

Its is not easy to directly understand an ER image.  Folks prefer to see a pin hole camera, or perhaps a fisheye lens view.  Below is the full ER image:

![im](Data/Images/equi_full.png)

And a virtual camera view extracted: a "mild" fisheye camera.  You can sort of tell it is fisheye because straight lines do not stay straight (like the backstay):

![im](Data/Images/equi_camera.png)

The process for constructing a "camera view" starts with a camera model.  The camera has a view direction, orientation, and field of view.  Each pixel in the camera can then be converted to a ray.  The lat/lon of this "view ray" can be looked up in the ER image and combined into a camera view.  The rays for pin hole cameras and fisheye lenses are both easily computed.

The webgl code to convert camera pixel coordinates to ER pixel coordinates is below.  This code is a **shader**, a function which is called for each pixel in the image to compute the pixel value (coordinates are passed as `gl_FragCoord.xy` the result is stuffed into `gl_FragColor`).  

The Fisheye lens transform and the "correction" below is described [here](http://paulbourke.net/dome/fisheyecorrect/fisheyerectify.pdf).  What makes a lens Fisheye is that the pixels at the outer border of the image are compressed (using `sin(phi)`).  In my experience for very wide views, a mild fisheye is much better than a pin hole camera.

```
     void main() {
         // Normalize and center to [-1, 1]
         // float scale = max(u_resolution.x, u_resolution.y);
         float scale = length(u_resolution);
         vec2 nc = (2.0 * gl_FragCoord.xy - u_resolution) / scale;
         nc = nc * vec2(1.0, -1.0); // Image texture is flipped vertically, flip back!!!

         // Going to construct a fisheye view with a third order correction of -0.2. 
         float correction = -0.2;
         float fov = radians(u_fov) / (1.0 + correction);

         // By convention, we follow the OpenGL framework for "right hand rule, x is horizontal,
         // y vertical, and looking down negative z".
         
         // First do the fisheye, converting image points to rays.
         // from http://paulbourke.net/dome/fisheyecorrect/fisheyerectify.pdf
         // Convert image coordinates to polar *image* coordinates and then transform there.
         float theta = atan(nc.y, nc.x);      // Orientation, angle from x axis.

         // The cubic "correction" reduces the "fish eye effect"
         float phi = (length(nc) + correction * pow(length(nc), 3.0)) * fov / 2.0;  // Distance from the center is view angle.

         // Convert back to a view ray.  
         vec4 ray = vec4(sin(phi)*cos(theta), sin(phi)*sin(theta), -cos(phi), 1.0);

         // Rotate the viewing direction.
         ray = u_rotation * ray;

         // Convert the rays to points in the equirectangular image.
         float lat = asin(ray.y)/M_PI + 0.5;
         float lon = atan(ray.x, -ray.z)/M_2_PI + 0.5;
         
         gl_FragColor = texture2D(u_texture, vec2(lon, lat));

     }
```

This WebGL code runs in the browser, computing the "virtual camera image" using `u_rotation` (the camera orientation/rotation), `u_fov` (the field of view), `u_texture` (the current equirectangular image).

## Getting this to work in the Browser

WebGL is a high performance scheme for creating complex graphics in the browser (a simplification of OpenGL).   It took me a long time to wrap my head around how this worked.  Several resources were invaluable.

- [WebGL Fundamentals](https://webgl2fundamentals.org/)
  - Awesome site with lots of great documentation and examples
- [ShaderToy](https://www.shadertoy.com/)
  - Great way to try out shaders in the brower with zero effort.
  - [Example on Equirectangular Images](https://www.shadertoy.com/view/4lK3DK)
- [TWGL](http://twgljs.org/) TWGL: A Tiny WebGL helper Library
  - WebGL web pages have a ton of fiddly little boilerplate.  This makes it much simpler.
- [glMatrix](https://github.com/toji/gl-matrix) library of vector/matrix/quaternion code in javascript which is compatible with WebGL.
- [Mozilla Developer Network: MDN](https://developer.mozilla.org/en-US/) a great collection of documentation on HTML, javascript, and how to program web pages.

## High Level Code Summary

The core of the web page works as follows.

- Open the video file using the HTML video element.  Play.
- Rather than rendering the video to the screen (as is typical) the video is used as the input to the WebGL shader (above).
- The shader creates and displays a virtual camera image which is displayed in an HTML canvas on the screen.
  - Direction of view, orientation, and field of view can be adjusted.
- If the camera poses are available, the virtual camera is rotated to keep the camera stable (mostly so that as the boat heels to port or starboard the "camera" still sees a level horizon).
- If instrument data is available display that as well.


# Stabilizing Camera Viewpoint

Code described below is in the directory VirbPose.  See README.md there as well. Code is in `VirbPose/virb_pose/virb_pose.cpp`.

Summary:  virb_pose reads a FIT file and creates one binary pose file for each video segment captured.

Code reads Garmin Virb 360 FIT files, and processes the gyro and accel data to create
a camera pose (orientation) estimate.  Results are written to a binary file as a set
of (timestamp, quaternion) pairs, intended to be rapidly read into the viewer app (web page).  The poses can be used to counter-rotate the virtual viewpoint, stabilizing the view.

This code also extracts key timing information, including the start and end of video capture.  Note, 
long VIRB videos are written as multiple video files (each about 30 mins).  Most often
a single fit file describes camera poses for multiple video files.  The FIT file
contains millisecond accurate info about the start and end of each video file (the file
creation times stored in the video files are close but not precisely the same).

The poses for each video file are written to distinct binary POSE files, each given a filename
which is the date/time of the video start.  This filename also provides means to accurately determine the start/stop time for each video.

The general approach used to estimate camera pose is the "complementary filter".
Watch this awesome video to see how it works: https://youtu.be/whSw42XddsU

The basic idea is that gravity, as measured by the accelerometers in the camera,
determines the down direction.  Note, this assumes that the camera is *not* in an
accelerating reference frame for a long period of time.  A well known counter example
is a "coordinated turn" in flight, where gravity does not point down but "down and out"
(but this never happens on a boat).  Unfortunately, the accelerometers are noisy, and sensitive to
vibration, this makes the "raw down" vector noisy as well.  It is far better to
average/smooth "gravity down" over a period of time.  But this makes the "down vector"
slow to respond.  

In addition to the accelerometers the Virb also has 3 gyros (a full fledged IMU).  The gyros directly measure the rate of rotation but do not measure absolute orientation.  The gyros are fast, and low noise.  The gyro information can be used to rapidly update the "down vector", while still relying on the accelerometers to estimate down over long time periods.

The linked video above does this all in angle space, which gets complex in 3D.  The code below
works in vector space through the magic of cross products and quaternions.

An additional note on stabilization.  Gravity can only tell us the down direction, not
the north or east direction.  One option would be to try to keep the compass heading of
the viewpoint constant (the VIRB camera does have a compass a.k.a magnetometer).  But this
is not great on a boat, where we would rather be looking (by default) in the direction
of travel (which rotates as the boat steers the course).  Call this direction forward.  
There is no way to directly measure local north, so we assume that the camera
has been mounted so that forward is along the boat's axis.

The complementary filter has an additional term that corrects the camera orientation so that
it points forward.




# Combining Videos, with Poses and Instrument Logs


Code to perform this is in `virb_video_pose.py`

Garmin Virb 360 produces two types of a files: MP4 videos and FIT metadata.  In a
given video "recording" there is a single FIT file, while videos are broken into 30 min chunks
(the FIT file contains annotation showing the precise start and end of each video).  The
FIT file is processed to compute the camera orientations (poses) associated with each video file, which are then
stored in separate binary pose files.

In addition, instrument log data may have been captured using the raspberry PI and stored in pandas (an
entirely separate system running on a different clock).  The clocks can be compared (in
most cases) because both are synced to GPS.

One simple scheme for aligning all this is to use the video file as the KEY with an associated (and optional) POSE and LOG file.  Note, there are times where it would be better to organize by "race" or by "day",  my plan is to layer this on top of the video based system.

Note, each video filename has two parts, and neither is the timestamp:  V1080460.MP4
    - File is part of the 108th **video** captured (which may have multiple parts like: V1080461.MP4, etc.)
    - File is 460th video **file** captured.  This is unique as well.

The video file creation times are encoded in the MP4 header, which is extracted by
exiftool (and stored in videos.json).  Video files appear to be created **after** the beginning of
the capture (stored in the FIT file).  So the pose two timestamps are not identical.  The videos files are typically created 27ish
seconds after the pose file begins.

    exiftool -api largefilesupport=1 -createdate -json *.MP4 > videos.json

First task: given a set of video files and pose files, find the correct association by
timestamp.

The pose file timestamp is encoded in its filename.  Pose files data and timestamp are
extracted from the FIT file.  These times are assumed to be precise (milliseconds are
encoded).

Similar to a FIT file, there is a single instrument log for the entire day.  The first
task is to find the correct log for each video.  Each video then refers to a slice of the
log, and this slice is extracted and stored in a binary log file.  The binary log files are stored using the same name as the pose file (since this is the
precise time encoding) and placed in a different directory.

The videos.json file is updated to include the name of the pose file and if it exists, the instrument log file.  The results are written to video_poses_logs.json 



## Other Steps in this process

- Instrument data is collected using a Raspberry PI
  - Data is processed to cleanup and normalize
  
- Video files and FIT files are captured using the Virb.
  - Pull the memory card and mount in Mac.
  - Script copies files.
  - On the camera, long videos are automatically split into 30 min files.  
  - Each video name has two parts:  V1080460.MP4
    - Video is 108th captured (which may have multiple parts like: V1080461.MP4, etc.)
    - Video is 460th file captured.  This is unique as well.
    - Times are not encoded in the name.

- Video creation times are extracted from exiftool
  - `exiftool -api largefilesupport=1 -json -createdate *.MP4 > videos.json`
  
- Poses are extracted from the FIT files using a virb_pose
  - 


## Out of date, but still informative, background

Read the chapter on [Garmin Fit Files](Garmin_Fit_files.ipynb) if you'd like additional info on FIT files.

Garmin is a pioneer in personal fitness, marine instruments, and action cameras.  Most Garmin devices produce FIT files which capture data from sensors, and this has led to a cool additional functionality: **you can overlay FIT data onto a video** (see image above).  This "dashboard" is created using a video editing tool from Garmin called [Virb Edit](https://buy.garmin.com/en-US/US/p/573412).  Virb is the brand name for Garmin cameras, and Virb Edit is used to edit video and produce final version.  Virb Edit can also be used to merge the video and the FIT data, producing awesome overlays.

Virb Edit calls FIT data "GMetrix" (for "Garmin Metrics"?).  There are a wide range of "instrument" types.  And you can put together a pretty professional video.  Virb Edit can be used with any video source (above is video from a GoPro).  

The FIT file format is a mostly public standard, except for one **super annoying** gap: the marine data like that shown above (e.g. AWA, TWA, etc) is **not part of the standard** (more on that below).  The data I used to discover this came from a Garmin boat (a J/99 called *One Life*).   Virb Edit can read this FIT file, and will overlay the encoded info on the video (even though it is unrelated to my boat).  Ultimately the lack of documentation has forced the development of an alternative solution that does not require FIT files.

If you search the web for applications that can produce videos with gauge overlays you find:

- [DashWare](http://www.dashware.net/) which is a flexible application that can produce videos with quality gauge overlays of many kinds (it claims to read *many* types of files).  
  - It is windows only.  
  - It can export video in "green screen" which can then be blended with other videos.
  - The goals of Dashware are generic, but it seems to mostly support auto racing.
  - Does not support 360 video directly (more on what this means below).
  - Looks fairly old and perhaps lightly supported.  There is some mention that these folks moved to GoPro.
  - Does not seem to read the FIT files.  Some mention of this on the web, but not much.

- [RaceRender](https://racerender.com/Products/index.html) this looks much like DashWare, but:
  - Has a Mac version.
  - Perhaps more actively maintained.
  - Seems very focused on automotive.
 
- Final Cut Pro X (from Apple for Mac).  Like a lot of professional tools, seems to have a steep learning curve.  But has support for 360 video (though I don't see how to incorporate Garmin FIT data).

- Adobe After Effects seems to have the ability to import and overlay mgJSON files which define the gauges.  There is a company that will automatically process GoPro video, extract the GPS and other data, and then produce the mgJSON files [GoPro Telemetry Extractor](https://goprotelemetryextractor.com/).
  - Commercial software (not a huge issue, but annoying).
  - I am sure mgJSON is well documented somewhere.  But it will require some deep diving.

- Garmin Virb Edit
  - Very nice end to end integration with Garmin systems (which generate FIT files).
    - If a Garmin camera is paired with a Garmin marine "plotter" it will record instrument data in addition to video.  (In a sense this replaces the Raspberry PI.)
    - This data is easily read by Virb Edit and can be overlayed with a set of standard gauges.
  - It can read FIT files (which can store lots of data) or GPX files (which just store lat/lon/alt).
  
Note, one critical feature is time alignment. Frequently video does not have timestamps.  In this case you need to align the instrument data with the video.  Both DashWare and Virb Edit have good alignment tools (though it is done by hand).


## Tooling

### Exiftool

> ExifTool is a platform-independent Perl library plus a command-line application for reading, writing and editing meta information in a wide variety of files. ExifTool supports many different metadata formats including EXIF, GPS, IPTC, XMP, JFIF, GeoTIFF, ICC Profile, Photoshop IRB, FlashPix, AFCP and ID3, as well as the maker notes of many digital cameras

[LINK](https://exiftool.org/)

- Important for very large files.

    exiftool -api largefilesupport=1 foo.mp4 
    
- To ensure 360 video is interpreted correctly

`exiftool -ProjectionType="equirectangular" -Spherical="true" baz.mp4 `
    
- The date *tags* file can be different from file date.

    exiftool V0130025.MP4  | grep -i date

```
    File Modification Date/Time     : 2020:04:09 21:41:24-07:00
    File Access Date/Time           : 2020:04:10 22:04:06-07:00
    File Inode Change Date/Time     : 2020:04:10 17:46:22-07:00
    Create Date                     : 2020:04:09 21:41:39
    Modify Date                     : 2020:04:09 21:41:39
    Track Create Date               : 2020:04:09 21:41:39
    Track Modify Date               : 2020:04:09 21:41:39
    Media Create Date               : 2020:04:09 21:41:39
    Media Modify Date               : 2020:04:09 21:41:39
```




### Video Overlays

In order to create awesome overlayed videos you need:  i) create a FIT file from the logs currently collected on the boat; ii) get that FIT file to load correctly into Virb Edit; iii) create widgets (gauges) which can display the required info.

There are two points of reference that have informed my design goals.


#### Reference 2: [FlightData Manager](https://sites.google.com/site/pud2gpxkmlcsv/)

FDM is a very nice package, that includes a software tool to post-process drone data to create FIT files which can be loaded into Virb Edit and will produce custom overlays.  FDM also includes a set of custom gauges that display drone info very well.

![im](Data/Images/flight_data_manager_example.jpg)

The author of FDM, Kenth Fuglsang Jensen, had many of the same goals that I have.  He collects video from his Parrot drone (which creates PUD files for telemetry).  He then needed to convert this to a FIT file.  Many of the fields in the drone data are missing or unsupported by Virb Edit (e.g. WIFI strength, and others).

Kenth did this in several steps.

- Figure out how to read the PUD files (we don't need this!).
- Through trial and error finding which messages and fields are supported by Virb Edit and which are not.
- Create new widgets (and templates) which can display this new information in useful ways.

**So I am building a FlightData Manger for sailboats.**